## Prepare data reader and model execution context

In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.markers import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "refined_traces"


# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name])
blockbatch_request = BlockBatchRequest.build(
    chains=["op"],
    range_spec="@20241118:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, aux_views = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)


2025-01-09 21:36:57 [debug    ] connecting to GOLDSKY Clickhouse client... counter=001/001 eta=None filename=client.py lineno=36 process=48946
2025-01-09 21:36:57 [info     ] loaded vault from .env file    counter=001/001 eta=None filename=vault.py lineno=32 process=48946
2025-01-09 21:36:57 [debug    ] loaded vault: 18 items         counter=001/001 eta=None filename=vault.py lineno=76 process=48946
2025-01-09 21:36:57 [debug    ] initialized GOLDSKY Clickhouse client. counter=001/001 eta=None filename=client.py lineno=41 process=48946
2025-01-09 21:36:57 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=36 process=48946
2025-01-09 21:36:58 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=41 process=48946
2025-01-09 21:36:58 [error    ] Encountered exception: HTTPDriver for https://qeyct17r6o.us-central1.gcp.clickhouse.cloud:8443 received ClickHouse error code 60
 Code: 60. DB::Exception: Unknown table expression identifier 'etl_

DatabaseError: HTTPDriver for https://qeyct17r6o.us-central1.gcp.clickhouse.cloud:8443 received ClickHouse error code 60
 Code: 60. DB::Exception: Unknown table expression identifier 'etl_monitor.blockbatch_markers' in scope SELECT dt, chain, marker_path, data_path, root_path, num_parts, num_blocks, min_block, max_block FROM etl_monitor.blockbatch_markers WHERE (dt IN _CAST([20045], 'Array(Date)')) AND (chain IN _CAST(['op'], 'Array(String)')) AND (root_path IN _CAST(['ingestion/blocks_v1', 'ingestion/traces_v1', 'ingestion/transactions_v1'], 'Array(String)')). (UNKNOWN_TABLE) (version 24.8.1.10545 (official build))


## Execute the model

It's up to you how the model manipulates the data. 

Develop the various steps in notebook cells and then copy the final code over to the model function.

In [2]:
refined_txs_table = aux_views["refined_transactions_fees"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_blocks": input_datasets["ingestion/blocks_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
    }
)

refined_traces_projection = aux_views["refined_traces/traces_projection"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": input_datasets["ingestion/traces_v1"].as_subquery(),
    },
)

traces_with_gas_used = aux_views["refined_traces/traces_with_gas_used"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "refined_traces_projection": refined_traces_projection,
    },
)

traces_txs_join = aux_views["refined_traces/traces_txs_join"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "traces_with_gas_used": traces_with_gas_used,
        "refined_transactions_fees": refined_txs_table,
    },
)

2025-01-08 15:02:36 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=19105
2025-01-08 15:02:36 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=19105
2025-01-08 15:02:36 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=19105 template=refined_transactions_fees
2025-01-08 15:02:40 [info     ] duck db size: 12.3KB           filename=client.py lineno=36 process=19105
2025-01-08 15:02:40 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=245 process=19105
2025-01-08 15:02:40 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=19105 template=refined_traces/traces_projection
2025-01-08 15:02:43 [info     ] duck db size: 14.2MB           filename=client.py lineno=36 process=19105
2025-01-08 15:02:43 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=19105 templa

In [5]:
ctx.client.sql("""
SELECT * FROM refined_traces__traces_txs_join
WHERE transaction_hash = '0x348e71e1290557d15582057f46562e1b05b6ab821d9e80015b2cd186b622ed47'
ORDER BY trace_address
LIMIT 10
""").show(max_rows=100)

┌────────────┬─────────┬──────────┬─────────┬──────────────┬─────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────────┬────────────────┬───────────────┬────────────┬───────────┬────────────────────┬─────────────────┬───────────────┬─────────────┬──────────────────────┬───────────────────┬────────────────────────┬──────────────────────────┬──────────────────────────────────┬───────────────────────────────────────┬───────────────────────────────────────────┬──────────────────────────────────────────────┬──────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬─────────────────────────────────────┬─────────────────────────────────┬────────────────────────────────────┬────────────────────────────────────┬───────────────────────────

In [9]:
ctx.client.sql(f"""DESCRIBE {refined_txs_table}""").show(max_rows=100)

┌─────────────────────────────────────┬────────────────┬─────────┬─────────┬─────────┬─────────┐
│             column_name             │  column_type   │  null   │   key   │ default │  extra  │
│               varchar               │    varchar     │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────┼────────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                                  │ DATE           │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                               │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id                            │ INTEGER        │ YES     │ NULL    │ NULL    │ NULL    │
│ network                             │ VARCHAR        │ YES     │ NULL    │ NULL    │ NULL    │
│ nonce                               │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_index                   │ BIGINT         │ YES     │ NULL    │ NULL    │ NULL    │
│ from_address                